# **WARNING**

In order for this notebook to run, you must first run the notebook `0_0_generate_skill_models.ipynb` and generate the transition probability distributions that are needed to evaluate the win probability Markov Chain. 

# 1.0 Notebook Parameters

Please specify the following variables:
1. `project_directory`: your project directory path
2. `epsilon_list`: the list of execution error multiplers (epsilon) that you want to consider in your analysis.
3. `player_id`: the players (i.e. skill model) you want to consider in your analysis; use one base model and scale it with different epsilons; this should match whatever was used in `1_0_generate_noturn_data.ipynb`.

In [6]:
# Change this to the project directory path for your computer  
project_directory = '/Users/rachael/Desktop/darts-thesis'

# Set the epsilons that will be considered in this analysis 
epsilon_list = [1,2,3,4,5,6,7,8,9]

# Set the ID of the player skill models you want to use 
    # 10 - symmetric DB skill model that we were using up until now 
    # "AVG" - "average pro" skill model generated by H & W 
player_id = 10 

# 2.0 Notebook Setup

Import libraries and run prerequisites.

In [16]:
# Import and configure modules from this project

import os

os.chdir(project_directory)

import function_init_simple_mdp as imdp
import function_init_board as fb
import class_handicap as hc

import function_helpers_singleplayer as h 
from function_helpers_twoplayer import evaluate_win_probabilities, build_win_probability_dataset

data_parameter_dir = fb.data_parameter_dir
result_dir = './result/twoplayer_ns_results/'  

handicap = hc.Handicap(epsilon_list=epsilon_list,results_file_name='result/singlegame_results/player10_turn_results.csv')

In [5]:
# Import and configure external modules

import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=300)
np.set_printoptions(threshold=300)

import pandas as pd


# 3.0 Generate Win Probability Dataset

### **WARNING**

This process takes a lot of RAM and produces very large data files. Each .csvs created below will be in hundreds of megabytes to a couple of gigabytes in size. It is recommended that you reset your RAM before trying to run the final cell which merges all of the datasets together into one single large file. 

The final file is about 13GB in size for a single player archetype and considering all weaker-stronger combinations for epsilons 1-9. 

In [13]:
# Part 1 - fetch the result dictionaries from the solved singleplayer non-strategic models

name_p = 'player{}'.format(player_id)

[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

result_dictionaries = {}

epsilon_list = range(1,10)

for e in epsilon_list: 

    [aiming_grid, prob_grid_normalscore_nt_a, prob_grid_singlescore_nt_a, prob_grid_doublescore_nt_a, prob_grid_triplescore_nt_a, prob_grid_bullscore_nt_a] = h.load_aiming_grid(name_p, epsilon=e, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

    result_dictionaries[e] = h.solve_dp_turn_tokens(9, aiming_grid, prob_grid_normalscore_nt_a, prob_grid_singlescore_nt_a, prob_grid_doublescore_nt_a, prob_grid_triplescore_nt_a, prob_grid_bullscore_nt_a, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t)



load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/t_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e1_gaussin_prob_grid_custom_no_tokens.pkl


/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:761: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:761: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:799: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:799: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:1218: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max]
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:1218: Runt

solve prob_policy_transit in 2.9804248809814453 seconds
solve dp_turn_policyiter in 32.51751899719238 seconds
[[0.     0.     1.8856 ... 5.5944 5.5758 5.6116]
 [0.     0.     1.     ... 4.9684 4.9631 4.9765]
 [0.     0.     1.     ... 4.7444 4.7403 4.7516]
 ...
 [0.     0.     1.     ... 3.8073 3.8027 3.8082]
 [0.     0.     1.     ... 3.5561 3.5447 3.5554]
 [0.     0.     1.     ... 3.     3.     3.    ]]
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e2_gaussin_prob_grid_custom_no_tokens.pkl
solve prob_policy_transit in 3.876865863800049 seconds
solve dp_turn_policyiter in 36.159262895584106 seconds
[[0.     0.     2.5371 ... 7.405  7.4122 7.4278]
 [0.     0.     1.     ... 6.2669 6.2766 6.2896]
 [0.     0.     1.     ... 5.8664 5.8755 5.8889]
 ...
 [0.     0.     1.     ... 3.9983 3.9972 4.0014]
 [0.     0.     1.     ... 3.7246 3.6864 3.6893]
 [0.     0.     1.     ... 3.     3.     3.    ]]
load_pickle from ./data_parameter/player_gaussin_fit/g

In [ ]:
# Part 2 - for each stronger player, solve the matches against all weaker players and store in .csv files

columns = ['epsilon_weaker','epsilon_stronger','score_weaker','score_stronger','credits_weaker','current_turn','win_probability_weaker']

for e_stronger in epsilon_list: 

    df = pd.DataFrame(columns=columns)

    print(f"Running for stronger player with epsilon={e_stronger}.")

    for e_weaker in epsilon_list: 

        if e_weaker >= e_stronger: 

            result_dic_weaker = result_dictionaries[e_weaker]
            result_dic_stronger = result_dictionaries[e_stronger]
            value_weaker, value_stronger = evaluate_win_probabilities(result_dic_weaker,result_dic_stronger)

            df = pd.concat([df,build_win_probability_dataset(e_stronger, e_weaker, value_weaker, value_stronger)])
            df = df.reset_index().drop(['index'],axis=1)

    file_title = result_dir + f'player{player_id}_win_probability_results_stronger_e{e_stronger}.csv'
    df.to_csv(file_title)

    del df

Running for stronger player with epsilon=1.
Running for stronger player with epsilon=2.
Running for stronger player with epsilon=3.
Running for stronger player with epsilon=4.
Running for stronger player with epsilon=5.
Running for stronger player with epsilon=6.
Running for stronger player with epsilon=7.
Running for stronger player with epsilon=8.
Running for stronger player with epsilon=9.


In [16]:
# Part 3 - combine all datasets into one master dataframe 

# NOTE: May need to restart the notebook and then run this cell and downwards. 

columns = ['epsilon_weaker','epsilon_stronger','score_weaker','score_stronger','credits_weaker','current_turn','win_probability_weaker']

df = pd.DataFrame(columns=columns)

for e in epsilon_list:
    print(e)
    file_title = result_dir + f'player{player_id}_win_probability_results_stronger_e{e}.csv'
    df = pd.concat([df,pd.read_csv(file_title)])

1
2
3
4
5
6
7
8
9


In [17]:
file_title = result_dir + f'player{player_id}_win_probability_results.csv'
df.to_csv(file_title)
df

,epsilon_weaker,epsilon_stronger,score_weaker,score_stronger,credits_weaker,current_turn,win_probability_weaker,Unnamed: 0
0,1,1,2,2,0,weaker,0.680422,0.0
1,1,1,2,2,0,stronger,0.319578,1.0
2,1,1,2,3,0,weaker,0.704062,2.0
3,1,1,2,3,0,stronger,0.369911,3.0
4,1,1,2,4,0,weaker,0.589241,4.0
...,...,...,...,...,...,...,...,...
4999995,9,9,501,499,9,stronger,1.000000,4999995.0
4999996,9,9,501,500,9,weaker,1.000000,4999996.0
4999997,9,9,501,500,9,stronger,1.000000,4999997.0
4999998,9,9,501,501,9,weaker,1.000000,4999998.0
